# Stream Ciphers: Concepts, Implementation, and Cryptanalysis

## Introduction

Cryptography is key to protecting digital communication, and stream ciphers are commonly used for their speed and efficiency, especially in systems with limited processing power. These ciphers encrypt data one bit at a time using a keystream, which is generated by pseudorandom number generators like Linear Feedback Shift Registers (LFSRs). While LFSRs are efficient, their linear nature can be a weakness. The Berlekamp-Massey Algorithm, for example, can reconstruct the shortest LFSR that produces a given output, making stream ciphers vulnerable to Known-Plaintext Attacks (KPA). To address these weaknesses, more complex designs like the Alternating Step Generator (ASG) are used. The ASG combines multiple LFSRs in a way that adds irregularity and non-linearity to the keystream, making it harder to analyze or predict [1].

In this assignment, 3 classes were implemented to further demonstrate the concepts mentioned above; the Bits class, the LFSR class, and the AlternatingStep class, representing the ASG. Additionally, the Berlekamp-Massey algorithm was implemented as a function to illustrate the KPA. This report explains the core components involved — including bit handling, LFSR construction, the Berlekamp-Massey Algorithm, and the ASG — to show how these systems work and where their strengths and weaknesses lie [2].


## Stream Ciphers


Stream ciphers are symmetric key algorithms that encrypt data one bit or byte at a time, offering a faster and smaller alternative to block ciphers. Their  design is inspired by the One-Time Pad (OTP), introduced by Gilbert Vernam in 1917 and later formalized by Shannon [1]. According to the OTP, the key must be as long as the plaintext, uniformly distributed in the key space, and must be used only once to achieve perfect secrecy, information theory-wise. In both the OTP and modern stream ciphers, encryption and decryption follow the same procedure by applying the XOR operation between the plaintext (or ciphertext) and the keystream.

Key streams are generated using random number generators (RNGs), which must be both reproducible and unpredictable. Because true randomness found in nature is not reproducible, practical systems rely on pseudorandom number generators (PRNGs) instead to create keystreams [3]. A simple and efficient PRNG used in stream ciphers is the Linear Feedback Shift Register (LFSR), although its linear structure can make it vulnerable to certain types of attacks.

In [1]:
from bits import Bits
from lfsr import LFSR, berlekamp_massey
from bitgenerator import AlternatingStep

## LFSR

one of the main building blocks of PRNGs

Linear Feedback Shift Registers (LFSRs) are simple but powerful components often used to generate pseudorandom sequences in stream ciphers. An LFSR consists of a series of binary registers (bits) that shift their values at each step, with the new input bit determined by a linear function, typically the XOR of selected bits from the current state, defined by a feedback polynomial [4]. Despite their simplicity, LFSRs can produce sequences with long periods and good statistical properties.

The output sequence of an LFSR can also be described by a generating function $G(x)$, which expresses the infinite series of output bits. Mathematically, $G(x)$ can be written as the quotient of two polynomials:

$$
G(x) = \frac{Q(x)}{P(x)}
$$

where $P(x)$ is the feedback polynomial defining the LFSR structure, and $Q(x)$ depends on the initial state of the register [4].


In stream ciphers, LFSRs act as keystream generators, where their output is XORed with the plaintext to produce ciphertext. The same sequence can be reproduced during decryption, as long as the initial state (or seed) and polynomial are known. However, the linear nature of LFSRs also makes them vulnerable. If an attacker can observe enough of the keystream, algorithms like Berlekamp-Massey can reconstruct the LFSR’s feedback polynomial and internal state, allowing full prediction of the remaining sequence. Regardless, they are the foundation for more complex and secure systems such as combination generators or the ASG.
The code below shows a sample generated keystream for the feedback polynomial $x^3 + x + 1$.


In [ ]:
#generate the first 14 bits of the keystream with the given feedback polynomial
poly = {0, 1, 3}
lfsr = LFSR(poly)
keystream = lfsr.run_steps(14)    
print("Generated keystream:", keystream)

Generated keystream: 11010011101001


## Berlekamp-Massey Algorithm

The Berlekamp-Massey Algorithm is a key tool in analyzing and attacking stream ciphers that use Linear Feedback Shift Registers (LFSRs). Originally developed for decoding linear error-correcting codes, the algorithm finds the shortest LFSR capable of generating a given binary sequence [2]. This means that if an attacker can observe a long enough portion of the keystream, they can use Berlekamp-Massey to reconstruct the LFSR's feedback polynomial and initial state, allowing them to recover the rest of the keystream.

The algorithm works by exploiting the property that any valid LFSR-generated sequence must satisfy the condition 
$\sum p_i \cdot b_{t-i} = 0$, 
where $p_i$ are the feedback polynomial coefficients and $b_{t-i}$ are the known bits of the sequence. At each step, the algorithm checks for a discrepancy — a violation of this condition — and updates the feedback polynomial accordingly by combining it with a shifted version of a previous one. This process continues until the full sequence has been processed, and the minimal LFSR capable of producing that sequence is returned [2]. Due to this capability, stream ciphers using LFSRs are vulnerable to Known-Plaintext Attacks (KPA), where an attacker, knowing parts of the plaintext and ciphertext, can recover the keystream and infer the generator.

In [13]:
# read the binary sequence from the file
with open('binary_sequence.bin', 'rb') as f:
        binary_sequence = f.read()
        
binary_sequence[:50]

b'\xbb`\xef\x067\xae\xd0K"Vd]#Q\xeb\x02~<\xe6C\xbe\xed5\xd0\xec\xada\xe8\x89h\xf3\xbdFc\x96\xb5\x8e\xb0\x03\xabVFY#\xd1\xeb">\xb5\xe4'

In [14]:
# convert the binary sequence to a Bits object
bits = Bits(binary_sequence)

In [15]:
# find the shortest feedback polynomial using the Berlekamp-Massey algorithm
poly = berlekamp_massey(bits)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 18, 7}
Linear complexity: 18


## Alternating Step Generator

The Alternating Step Generator (ASG) is a stream cipher design that aims to overcome the weaknesses of individual LFSRs by combining multiple LFSRs in a non-linear and irregular fashion [5]. In a typical ASG setup, three LFSRs are used: two generate bits for output, while the third acts as a control register that determines which of the other two is clocked at each time instant. This irregular clocking introduces unpredictability into the output sequence, making it harder for an attacker to analyze or reverse-engineer. The output is then the XOR of the 2 LFSRs.

The core idea behind the ASG is to break the linearity that makes LFSRs vulnerable to attacks like Berlekamp-Massey. By conditionally stepping LFSRs based on the output of a third, the generator produces a more complex and less predictable keystream, even if the individual components are linear. This added complexity significantly raises the difficulty of reconstructing the internal state through known cryptanalytic techniques [5]. Still, while ASGs are more secure than a single LFSR, they aren’t foolproof. If the parameters aren’t chosen carefully or if there isn’t enough non-linearity in the design, they can still be exposed to more advanced or targeted attacks.

In [ ]:
#generate the first 20 bits of the keystream with the alternating step generator
asg = AlternatingStep()  
keystream = asg.run(20) 
print("Generated keystream:", keystream)

Generated keystream: 00011101010111000100


# KPA

A Known-Plaintext Attack (KPA) targets stream ciphers by using knowledge of part of the original plaintext and its corresponding ciphertext to recover the underlying keystream. Since stream ciphers encrypt data by XORing the plaintext with a keystream, simply XORing the known plaintext with its ciphertext segment yields the matching portion of the keystream. The example below demonstrates this process.

In [19]:
# open the ciphertext and known plaintext files
with open("ciphertext.bin", "rb") as f:
    ciphertext = f.read()

with open("known-plaintext.txt", "r", encoding="utf-8") as f:
    known_plaintext = f.read().encode("utf-8")
    
print(f"Ciphertext: {ciphertext[:30]} ..., \nKnown-Plaintext: {known_plaintext[:90]}...")

Ciphertext: b"\xb7;\xcep\x9e\x7f\xc0\xe3H_'\xc6D\x9b\xe8\xbd\x8e[\x8b\xb0\x94\x00\xdf]\xa9\xd9\x152k\x06" ..., 
Known-Plaintext: b'The Legacy of the Hidden Key\n\nIn a quiet corner of the university library, where dust mote'...


In [42]:
# convert the binary sequences to a Bits objects
kp_bits = Bits(known_plaintext)
cipher_bits = Bits(ciphertext)

In [43]:
# using the known plaintext, find the polynomial that produces the bit sequence needed to generate the ciphertext
bit_sequence = kp_bits ^ Bits(cipher_bits[:len(kp_bits)])
poly = berlekamp_massey(bit_sequence)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 1, 9, 48, 19}
Linear complexity: 48


In [44]:
# recover the initial state of the LFSR using the polynomial and the first 48 bits of the bit sequence
init_state = Bits(bit_sequence[0:48][::-1])
lfsr = LFSR(poly, init_state)

In [45]:
# Generate the binary sequence for decryption using LFSR, starting with the initial state
decryption_bits = Bits([lfsr.output])
decryption_bits += lfsr.run_steps(len(cipher_bits)-1, init_state)

In [46]:
# decrypt the ciphertext using the generated binary sequence
decrypted_bits = decryption_bits ^ cipher_bits
decrypted_bytes = decrypted_bits.to_bytes()
print(decrypted_bytes[:130])

b'The Legacy of the Hidden Key\n\nIn a quiet corner of the university library, where dust motes danced in the slanted afternoon light,'


## Conclusion

Stream ciphers are a fast and efficient way to encrypt data, but their security depends on how unpredictable the keystream is. While LFSRs are easy to use and generate good sequences, their linear structure makes them vulnerable to attacks when part of the keystream is known. The Berlekamp-Massey Algorithm showed how the feedback polynomial of an LFSR can be recovered, exposing the weaknesses of simple designs.

The Known-Plaintext Attack carried out in this project demonstrated how recovering even a portion of the keystream can be enough to reconstruct the generator and decrypt the full message. This highlights the real-world risks of relying solely on linear structures for security. Adding complexity through methods like the Alternating Step Generator can make stream ciphers stronger, but even with improvements, careful design and parameter choices are necessary to protect against more advanced attacks. A good stream cipher must balance efficiency with strong resistance to cryptanalysis.

## References

[1] C. E. Shannon, Communication Theory of Secrecy Systems, Bell System Technical Journal, vol. 28, no. 4, 1949.

[2] J. L. Massey, Shift-Register Synthesis and BCH Decoding, IEEE Transactions on Information Theory, vol. 15, no. 1, 1969.

[3] J. Katz and Y. Lindell, Introduction to Modern Cryptography, 3rd ed. Boca Raton, FL, USA: CRC Press, 2020.

[4] A. J. Menezes, P. C. van Oorschot, and S. A. Vanstone, Handbook of Applied Cryptography, Boca Raton, FL, USA: CRC Press, 1996.

[5] S. W. Golomb, Shift Register Sequences, Revised ed., Laguna Hills, CA, USA: Aegean Park Press, 1982.